# Percepción (PER): examen de prácticas del bloque 1, grupo 3CO11, turno 2, 25-4-2023, 8:45-9:30

Lee este cuaderno y realiza las actividades y ejercicios propuestos.

## Importación de librerías relevantes

Como sabes, sklearn facilita el aprendizaje y evaluación de clasificadores discriminativos en digits. Ejecuta el código siguiente para importar librerías relevantes:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline

## Lectura del corpus digits

Ejecuta el código siguiente con random_state igual a las tres últimas cifras de tu DNI/NIE:

In [2]:
digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, test_size=0.75, shuffle=True, random_state=582)
print(X_train.shape, y_train.shape)

(449, 64) (449,)


## Perceptrón

Ejecuta el código siguiente para estudiar la precisión de Perceptrón con penalización L2 en función de alpha:

In [3]:
for a in np.logspace(-8,-3,6):
    clf = Perceptron(penalty='l2', alpha=a, random_state=0).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de Perceptron(alpha=1e-08, penalty='l2') es 90.0%
La precisión de Perceptron(alpha=1e-07, penalty='l2') es 90.0%
La precisión de Perceptron(alpha=1e-06, penalty='l2') es 91.7%
La precisión de Perceptron(alpha=1e-05, penalty='l2') es 91.5%
La precisión de Perceptron(penalty='l2') es 88.1%
La precisión de Perceptron(alpha=0.001, penalty='l2') es 75.5%


## Regresión logística

Añade una celda de código para estudiar regresión logística con solver lbfgs, en función del preproceso. Aparte de características polinómicas (de grado 2 a lo sumo), el preproceso de características puede incluir estandarización (StandardScaler) y reducción de la dimensión (PCA). Determina una configuración del preproceso que obtenga máxima precisión.

In [6]:
clf = LogisticRegression(solver='lbfgs', max_iter=10000).fit(X_train, y_train)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de LogisticRegression(max_iter=10000) es 94.5%


In [11]:
def lr_exp(standardize=True, degree=2, n_components=0):
    clf = make_pipeline(StandardScaler() if standardize else None, 
    PolynomialFeatures(degree=degree),
    PCA(n_components=n_components) if n_components > 0 else None,
    LogisticRegression(max_iter=10000, solver='lbfgs', n_jobs=8)).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)
for n_components in [8, 16, 32, 64, 128, 256, 449, 0]:
    for standardize in [False, True]:
        acc = lr_exp(standardize=standardize, degree=2, n_components=n_components)
        print('standardize {0:} components {1:}: {2:.1%} acc'.format(standardize, n_components, acc))

standardize False components 8: 85.5% acc
standardize True components 8: 58.4% acc
standardize False components 16: 90.9% acc
standardize True components 16: 79.0% acc
standardize False components 32: 93.2% acc
standardize True components 32: 87.8% acc
standardize False components 64: 94.9% acc
standardize True components 64: 94.4% acc
standardize False components 128: 95.6% acc
standardize True components 128: 96.6% acc
standardize False components 256: 96.0% acc
standardize True components 256: 97.2% acc
standardize False components 449: 96.1% acc
standardize True components 449: 97.3% acc
standardize False components 0: 96.1% acc
standardize True components 0: 97.3% acc


A la vista de los resultados, la forma optima de preproceso para el solver lbfgs y random.seed= 582, es con estandarizacion y con un numero de componentes 449 o bien 0, ya que obtienen la misma tasa de acierto, ambas dan 97.3%. Quizas en este caso, conviene elegir el n_componentes = 0 ya que emplea menos recursos computacionales y algoritmicamente será algo más ligero, pero en realidad da un poco igual. Tambien es cierto que si hacemos n_componentes=0, entonces evitamos la posibilidad de perder cierta información. Como se ve, es mejor en general preprocesar los datos que no hacerlo, en casi cualquier caso, porque obtenemos un 94.5% de precision en caso de no hacerlo. 